In [1]:
from uniswappy import *

user_nm = 'user0'
eth_amount = 10000
tkn_amount = 100000

fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [2]:
n_steps = 50
start_price = eth_amount/tkn_amount
mu = 0.3; sigma = 0.8
n_paths = 25

b = BrownianModel(start_price)
p_arr = b.gen_gbms(mu, sigma, n_steps, n_paths)
exp_p_arr = np.median(p_arr, axis = 1)

In [3]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", address="0x011")

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, tkn_amount)
lp.summary()

Exchange ETH-TKN (LP)
Reserves: ETH = 10000, TKN = 100000
Liquidity: 31622.776601683792 



In [4]:
arb = CorrectReserves(lp, x0 = 1/exp_p_arr[0])
for k in range(1, n_steps):
    p = 1/exp_p_arr[k]
    arb.apply(p)
    
    
    select_tkn = EventSelectionModel().bi_select(0.5)
    bias_factor = 1
    rnd_amt = TokenDeltaModel(30).delta()

    if(select_tkn == 0):
        out = Swap().apply(lp, eth, user_nm, rnd_amt) 
    else:
        out = Swap().apply(lp, tkn, user_nm,  bias_factor*p*rnd_amt)  
   
    print(f'price {lp.get_price(tkn)} {exp_p_arr[k]}')

print('')
lp.summary()

price 0.09924437740613659 0.0993422255273003
price 0.09245178387082753 0.09244363252384762
price 0.09431684029260416 0.0940756580014922
price 0.09336958373929048 0.0933701796016427
price 0.09136079066526395 0.09129534808965487
price 0.08710763644978164 0.08704194042028487
price 0.09094559490912585 0.09096847908364161
price 0.09480603943317015 0.0949953734008471
price 0.09917337649888418 0.09909786570892481
price 0.0953473985936487 0.09548968002163591
price 0.09374698211810109 0.0940601516826128
price 0.09804775986906868 0.09803620625728285
price 0.09625317569959052 0.09614298874728221
price 0.08507534399191984 0.08519319114307158
price 0.0891425434489485 0.0891397418378267
price 0.085916190552005 0.08590188793470459
price 0.08748033263837664 0.08728588431448923
price 0.09354090443537333 0.09364204165476567
price 0.08869434221491734 0.0888324371336298
price 0.08810077869486141 0.08812789000647787
price 0.08744833698563786 0.0875022342969827
price 0.07875295642186904 0.07871107316761305


In [5]:
eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, tkn_amount, lwr_tick, upr_tick)
lp.summary()

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 9999.999999999998, TKN = 100000.0
Liquidity: 31622.776601683792 



In [6]:
arb = CorrectReserves(lp, x0 = 1/exp_p_arr[0])
for k in range(1, n_steps):
    p = 1/exp_p_arr[k]
    arb.apply(p, lwr_tick, upr_tick)
    
    
    select_tkn = EventSelectionModel().bi_select(0.5)
    bias_factor = 1
    rnd_amt = TokenDeltaModel(30).delta()

    if(select_tkn == 0):
        out = Swap().apply(lp, eth, user_nm, rnd_amt) 
    else:
        out = Swap().apply(lp, tkn, user_nm,  bias_factor*p*rnd_amt)  
   
    print(f'price {lp.get_price(tkn)} {exp_p_arr[k]}')

print('')
lp.summary()

price 0.0994075363318794 0.0993422255273003
price 0.092490865344796 0.09244363252384762
price 0.09433583766475814 0.0940756580014922
price 0.09314126046905373 0.0933701796016427
price 0.0909572150611202 0.09129534808965487
price 0.08699780504410845 0.08704194042028487
price 0.091045615165147 0.09096847908364161
price 0.09503222333180718 0.0949953734008471
price 0.09878857142166587 0.09909786570892481
price 0.0950502852064888 0.09548968002163591
price 0.09398874245006252 0.0940601516826128
price 0.09776228008397915 0.09803620625728285
price 0.09613941706007456 0.09614298874728221
price 0.08515946540931656 0.08519319114307158
price 0.08925306322592573 0.0891397418378267
price 0.08613863014382188 0.08590188793470459
price 0.0873759688644783 0.08728588431448923
price 0.09357284503610898 0.09364204165476567
price 0.08890111875942877 0.0888324371336298
price 0.08811975916747733 0.08812789000647787
price 0.08747304087837529 0.0875022342969827
price 0.07877229022867978 0.07871107316761305
pric

In [7]:
lp.get_virtual_reserve(tkn)

104386.08683583206

In [8]:
lp.get_virtual_reserve(eth)

10053.607710128284